<a href="https://colab.research.google.com/github/yexf308/AdvancedMachineLearning/blob/main/Recommender_Systems_implicit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\def\m#1{\mathbf{#1}}$
$\def\mb#1{\mathbb{#1}}$
$\def\c#1{\mathcal{#1}}$

# Implicit feedback
We have assumed that the user gives explicit ratings for each item that they interact with.
This is a very restrictive assumption. More generally, we would like to learn from the **implicit feedback** that users give just by interacting with a system.

we can view the fact that they watched movie $i$ but did not watch movie $j$ as an
implicit signal that they prefer $i$ to $j$. 

Even when we do have explicit data, it might still be a good idea to incorporate implicit data into the model. Consider, for example, listening to songs. When users listen to music on a streaming service, they might rarely ever rate a song that he/she like or dislike. But more often they skip a song, or listen only halfway through it if they dislike it. If the user really liked a song, they will often come back and listen to it. So, to infer a user's musical taste profile, their listens, repeat listens, skips and fraction of tracks listened to, etc. might be far more valuable signals than explicit ratings.

<img src="https://github.com/yexf308/AdvancedMachineLearning/blob/main/image/implicit1.png?raw=true" width="300" />

### Possible method
- Treat all the missing entries as zeros and minimizing the loss on all the entries. 

- 95% elements are zero and it tend to fit zeroes instead of ones. 

<img src="https://github.com/yexf308/AdvancedMachineLearning/blob/main/image/implicit2.png?raw=true" width="500" />


- Or one can use the exact the same method in the explicit feedback. We make the assumption that if a user has interacted at all with an item, if the user $u$ has a preference for item $i$, then $P_{ui}=1$. However, when $P_{ui}=0$, we assume that it should be associated with a lower confidence, as there are many reasons beyond disliking the item as to why the user has not interacted with it,  e.g. Unaware of its existence.



## Method 1: Weighted matrix factorization 
The new objective function $L_{\text{imp}}(W,H)$:
\begin{align}
\min_{W\in \mb{R}^{M\times k}, H\in \mb{R}^{N\times k}}\sum_{u,i}c_{ui}(P_{ui}-W_u H_i^\top)^2  +\lambda (\|W\|_F^2 + \|H\|_F^2)
\end{align} 
where
$$ p_{ui}=\begin{cases} 1 & \text{if } A_{ui}>0 \\ 
0 & \text{Otherwise} \end{cases}$$
and $c_{ui}$ measures the confidence in observing $p_{ui}$. 

  -  It is important to transform the raw observation $A_{ui}$ into two distinct representation: the preference $p_{ui}$ and the confidence levels of the preference $c_{ui}$.
        - when $A_{ui}=0$, $p_{ui}=0$ can be either “negative” or “unobserved” ⇒ lower weight on $c_{ui}$

        - When $A_{ui}>0$, $p_{ui}=1$ is always “positive” ⇒ higher weight on $c_{ui}$

        - The larger $A_{ui}$ is, the higher weight $c_{ui}$ is and it indicates the higher confidence. 
   
  - $c_{ui}= 1+\alpha A_{ui}$ or $c_{ui}=1+\alpha \log(1+ A_{ui}/\epsilon)$
     - The parameter $\alpha$ controls the rate of the increase. 
     
     - A larger $c_{ui}$ means $W_u H_i^\top$ has to be that much close to $P_{ui}$ so that term will not contribute to the total loss. 



### Alternating Least Square

Assume $M$ users and $N$ items, hidden dimension k.

- Treat $H_i$ as constant and solve the loss function with respect to $W_u$. 
  - We can rewrite the loss function in the matrix form 
  \begin{align}
  \sum_{u,i}c_{ui}(P_{ui}-W_u H_i^\top)^2 &= \sum_u c_u \|P_u - W_u H^\top\|_2^2 \\
  &= \sum_u \left(P_u C^u P_u^\top - 2 W_u H^\top C^u P_u^\top + W_uH^\top C^u H W_u^\top \right)
  \end{align}
where $P_u\in \mb{R}^{1\times N}$ is the row vector has all the preference of the user. The diagonal matrix $C^u$ with the diagonal of $c_u$. i.e, $C^u = \text{diag}(c_u)$. 

  - If we set $\m{A}=H^\top C^u H $ and $\m{B}= H^\top C^u$, the equation above becomes 
   \begin{align}
   \sum_{u,i}c_{ui}(P_{ui}-W_u H_i^\top)^2 = \left(\sum_u P_u C^u P_u^\top\right) - 2W_u \m{B}P_u^\top + W_u \m{A} W_u^\top 
   \end{align}

  - For the partial derivative, 
  \begin{align}
  &\frac{\partial L_{\text{imp}}}{\partial W_u} = -2\m{B}P_u + 2\m{A}W_u +2\lambda W_u =0\\
  & (\m{A}+\lambda I)W_u =\m{B}P_u \\
  &\Rightarrow W_u^* =(\m{A}+\lambda I)^{-1}\m{B}P_u
  \end{align}
  where $\m{A}= H^\top C^u H$ and $\m{B}=H^\top C^u$. 


- Treat $W_u$ as constant and solve the loss function with respect to $H_i$.
   - Similarly, 
   \begin{align}
   H_i^* = (\m{D}+\lambda I)^{-1}\m{E}(P^\top)_i
   \end{align}
   where $\m{D}=W^\top C^i W$ and $\m{E}=W^\top C^i$.

- Several tricks to speed up the calculation:
   - $W^\top C^iW$ and $H^\top C^u H$ need to be calculated each $i$ and each $u$. This is very expensive. We will rewrite the expression as follows:
   \begin{align}
   &H^\top C^u H = H^\top H + H^\top (C^u-I) H\\
   &W^\top C^iW  = W^\top W + W^\top (C^i-I) W
   \end{align}  
   Note $ H^\top H$ and $W^\top W$ are independent of each user and each item, so they can be computed independently. $C^u-I$ has only $n_u$ nonzero elements, where $n_u$ is the number of items for which $r_{ui}>0$. Similarly, $C^uP_u$ contains only $n_u$ non-zero elements. 

   - computing matrix product $H^\top H$ using a outer product of each row. i.e., $H^\top H =\sum_{i=1} H_i^\top H_i$. The matrix product is now the sum of the outer products of the rows, where each row's computation is independent of another can be computed in the parallelized fashion then added back together. 

## Method 2: Matrix factorization with pairwise loss

- Idea: using pair-wise comparisons to design the loss function

  - If $A_{ui}=1$ and $A_{uj}=0$, then user $u$ prefer $i$ over $j$. 

  - Therefore $W_uH_i^\top$ should be larger than $W_uH_j^\top$.

- Matrix factorization with pair-wise loss: $f(W,H)$
\begin{align}
f(W,H) = \sum_{u=1}^M \left(\sum_{(i,j): A_{ui}=1, A_{uj}=0}\ell\left(W_u(H_i^\top - H_j^\top)\right)\right)
\end{align}
The optimization problem is 
\begin{align}
\min_{W\in \mb{R}^{M\times k}, H\in \mb{R}^{N\times k}} f(W,H)
\end{align}

- The choice of the loss function: Classification loss on $W_u(H_i^\top - H_j^\top)$. We want this to be positive. 

  - Hinge loss: 
     \begin{align}
     \ell\left(z\right) = \max(0, 1-z)
     \end{align}

  - Logistic loss (also called Bayesian Personalized Ranking, BPR)
    \begin{align}
     \ell\left(z\right) = \log(1+e^{-z})
     \end{align}
    
### Algorithm: SGD
- Each iteration sample $(u,i,j)$ by
   - sample a user $u$

   - sample a positive item $i$ ($A_{ui}=1$)

   - Sample a negative (unobserved) item $j$ ($A_{uj}=0$)

- The sampled loss function: 
 $$\ell(W_u, H_i, H_j)=\log(1+\exp(-W_u(H_i^\top - H_j^\top))) $$    


- Update $W_u, H_i, H_j$ by SGD:
  - $W_u\leftarrow W_u -\nabla_{W_u}\ell(W_u, H_i, H_j)$ 

  - $H_i\leftarrow H_i -\nabla_{H_i}\ell(W_u, H_i, H_j)$

  - $H_j\leftarrow H_j -\nabla_{H_j}\ell(W_u, H_i, H_j)$

